# Random Forest
## 범주형

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path="/content/drive/MyDrive/빅데이터 팀플/dementia_analytics/"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 데이터 로딩 ### 
train_dataset = pd.read_csv(path + 'data_processing/dataset_03/training/train_dataset_with_label_fill_user_mean.csv')
valid_dataset = pd.read_csv(path + 'data_processing/dataset_03/validation/val_dataset_with_label_fill_user_mean.csv')

train_output = train_dataset[['DIAG_NM']]
train_input = train_dataset[['sleep_breath_average', 'sleep_hr_average', 'sleep_hr_lowest', 'sleep_restless', 'sleep_midpoint_at_delta', 'activity_cal_total', 'activity_met_min_inactive', 'activity_met_min_high', 'activity_score_meet_daily_targets', 'sleep_rem', 'sleep_light', 'activity_low', 'activity_rest', 'activity_met_min_medium']]

test_output = valid_dataset[['DIAG_NM']]
test_input = valid_dataset[['sleep_breath_average', 'sleep_hr_average', 'sleep_hr_lowest', 'sleep_restless', 'sleep_midpoint_at_delta', 'activity_cal_total', 'activity_met_min_inactive', 'activity_met_min_high', 'activity_score_meet_daily_targets', 'sleep_rem', 'sleep_light', 'activity_low', 'activity_rest', 'activity_met_min_medium']]

In [ ]:
### 레이블 인코딩 ### 

# 트리 기반 알고리즘에서는 숫자의 크기에 따른 중요도 차이가 없어 레이블 인코딩 OK
# 선형적 특징을 가지는 알고리즘들은 one hot encoding 통해 해결해야 함

from sklearn.preprocessing import LabelEncoder

# 라벨인코더 선언 및 Fitting
# CN : 0, Dem : 1, MCI : 2
items = ['CN', 'Dem', 'MCI']
encoder = LabelEncoder()
encoder.fit(items)

# train
train_output['DIAG_NM'] = encoder.transform(train_output['DIAG_NM'])
# display(encoder.classes_)
# display(encoder.inverse_transform(train_output['DIAG_NM']))

# test
test_output['DIAG_NM'] = encoder.transform(test_output['DIAG_NM'])

<ipython-input-5-e819ad067ec6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_output['DIAG_NM'] = encoder.transform(train_output['DIAG_NM'])
<ipython-input-5-e819ad067ec6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_output['DIAG_NM'] = encoder.transform(test_output['DIAG_NM'])


In [ ]:
### RandomForestClassifier ### 

from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state=0, n_jobs=-1)

RF.fit(train_input, train_output)

print(RF.score(train_input, train_output))
print(RF.score(test_input, test_output)) # 0.6751873071837814

<ipython-input-6-b80fb5c6257e>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF.fit(train_input, train_output)


1.0
0.7036910457963089


In [ ]:
### RandomForest GridSearchCV ### 

from sklearn.model_selection import GridSearchCV

params ={
    'n_estimators':[10, 100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

RF = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(RF, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(train_input, train_output)
print(grid_cv.score(train_input, train_output))
print(grid_cv.score(test_input, test_output)) # 0.7236668135742618
print(grid_cv.best_params_) # {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 10}

0.6741354975299929
0.7624743677375256
{'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 10}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
